# E. Ejercicios laboratorio 2 MA4702-2021

Este laboratorio tiene 6 ejercicios.
La parte presencial consiste en los ejercicios 1 al 4 (idealmente comience los siguientes)
El resto se entrega el viernes 28 de mayo de acuerdo a instrucciones.

Observación:  
Los ejercicios 1; 2; 3a; 4b; 5a no tienen requisitos.  
Una vez tenga hecho 1, 2 y 3a puede hacer el resto de 3, 4 y 5.
Para hacer ejercicio 6 necesita todos los anteriores.lñ

Atrasos:
Cada día de atraso penalizará en 10 décimas su nota final de laboratorio
Con un máximo de 4 días de atrasos (luego de eso, tendrá un 1.0)


# 🔴 Datos grupo y Observaciones

Escriba en esta celda el nombre de cada integrante de su grupo.

- Integrante 1: Carlos Antil

- Integrante 2: Vicente Cabezas

- Integrante 3: Jorge Hewstone

- Integrante 4: Javier Santidrián

- Integrante 5: Arturo Lazcano


## Observaciones

- Si tiene alguna observación a la corrección escríbala acá.

In [1]:
#Ejecute esta celda una vez

using JuMP, Gurobi, DataFrames
const GUROBI_ENV = Gurobi.Env()

function instancia(i)
    p1 = [2,4,3]
    w1 = [5,1,3]
    A1 = [[3,1]]
    p2 = [3, 14, 4, 15, 11, 12, 12, 5, 15, 6, 13, 7, 11, 9, 5]
    w2 = [63, 41, 66, 23, 79, 23, 67, 56, 29, 74, 34, 51, 55, 42, 76]
    A2 = [[3,1],[5,7], [6,5],[10,1]]
    p3 = [9, 4, 5, 4, 12, 1, 2, 6, 1, 5, 11, 8, 2, 3, 9, 4, 11, 1, 5, 7, 8, 7, 12, 3, 2]
    w3 = [21, 8, 36, 84, 2, 3, 21, 40, 81, 92, 16, 78, 82, 49, 85, 9, 72, 35, 52, 74, 92, 36, 10, 54, 15]
    A3 = [[6, 9], [13, 17], [1, 21], [19, 24], [2, 17], [21, 22], [3, 7], [2, 25]]
    p4 = [7, 2, 2, 8, 1, 6, 1, 8, 9, 1, 2, 2, 11, 1, 11, 8, 5, 2, 8, 10, 1, 5, 5, 9, 8, 11, 3, 2, 12, 7, 7, 9]
    w4 = [95, 61, 77, 9, 35, 38, 66, 72, 84, 57, 83, 92, 35, 11, 52, 6, 92, 98, 51, 2, 72, 2, 70, 42, 38, 8, 73, 23, 20, 95, 31, 86]
    A4 = [[4, 17], [4, 26], [15, 27], [7, 32], [14, 29], [17, 22], [3, 26], [8, 13], [19, 20], [7, 16], [6, 26], [10, 27]]
    if i==1
        return p1,w1,[]
    elseif i==2
        return p1,w1,A1
    elseif i==3
        return p2,w2,[]
    elseif i==4
        return p2,w2,A2
    elseif i==5
        return p3,w3,[]
    elseif i==6
        return p3,w3,A3
    elseif i==7
        return p4,w4,[]
    elseif i==8
        return p4,w4,A4
    end
    @error "i debe estar entre 1 y 8"
    return 
end

## Ejemplo generador de precedencias
## using StatsBase
## sample([[i,j] for i in 1:20 for j in i+1:20], 8; replace=true)

Academic license - for non-commercial use only - expires 2021-07-24


instancia (generic function with 1 method)

## 🔴 Ejercicio 1. Modelos Originales

Nota: Para cargar una instancia puede escribir
```julia
p,w,A=instancia(i)
```

Escriba funciones ```creaModeloRect1(p,w,A)``` y ```creaModeloOrden1(p,w,A)```
     que retornen modelos tipo Rect1 (como en la sección B) y tipo Orden1 (como en la sección C) respectivamente 
para instancias con tiempos de proceso p, pesos w, y precedencias A.
     En ambas funciones, su modelo debe crearse con las opciones.
     
> TimeLimit: 15  
> Threads: 1  
> Presolve: 0  
> Heuristics: 0  
> Cuts: 0  

Al final debe probar su modelo, optimizando ambos en las instancias simples i=1, i=2, i=3, i=4. 
(Nota: Es posible que para i=3 e i=4 no encuentre solución, no se preocupe, pero revise que los óptimos coincidan en las intancias 1 y 2)



In [2]:
function creaModeloRect1(p,w,A)
    n =  length(p)
    J = [i for i in 1:n]
    T = sum(p[i] for i in J)
    modelo = Model(optimizer_with_attributes(
        () -> Gurobi.Optimizer(GUROBI_ENV),
        "TimeLimit" => 15,
        "Threads" => 1,
        "Presolve" => 0,
        "Heuristics" => 0,
        "Cuts" => 0))
    # Crear variables, objetivos y restricciones
    @variable(modelo, 0<=Q[i in 1:n,t in 1:T]<=1, Int)   
    
    @objective(modelo, Min, sum( sum( Q[i ,t]*t*w[i] for t in 1:T) for i in J ))
    for i in J
    @constraint(modelo, sum( Q[i, t] for t in 1:T) == 1 )
        for t in 1:T
            if t<= p[i]- 1
            @constraint(modelo, Q[i,t] == 0)
            end
        end
    end
    for i in J, j in J
        if i<j
            for s in 1:T, t in 1:T
                if t-p[j]+1<= s && s-p[i]+1 <= t
                    @constraint(modelo, Q[i,s]+Q[j,t]<=1)
                end
            end
        end
    end
    for s in 1:T, e in A
        @constraint(modelo, sum(Q[e[1], t ] for t in 1:s) >= sum(Q[e[2], t ] for t in 1:s))
        end
    return modelo
end

creaModeloRect1 (generic function with 1 method)

In [3]:
function creaModeloOrden1(p,w,A)
    n =  length(p)
    J = [i for i in 1:n]
    M = sum(p[i] for i in J)
    modelo = Model(optimizer_with_attributes(
        () -> Gurobi.Optimizer(GUROBI_ENV),
        "TimeLimit" => 15,
        "Threads" => 1,
        "Presolve" => 0,
        "Heuristics" => 0,
        "Cuts" => 0))
    # Crear variables, objetivos y restricciones
    @variable(modelo, 0<=y[i in J,j in J]<=1, Int)
    
    @variable(modelo, Z[i in J], Int)
    
    @objective(modelo, Min, sum(Z[i]*w[i] for i in J ))
    for i in J 
        @constraint(modelo, y[i,i] == 1)
        for j in J
            if i< j
                @constraint(modelo, y[i,j]+y[j,i] ==1)
            end
            for k in J
                if (i<j && j<k)|| (i<k && k<j )
                    @constraint(modelo, y[i,j]+y[j,k]+y[k,i] <=2)
                end
            end
            
        end
    end
    for e in A
        @constraint(modelo,y[e[1], e[2]] ==1)
    end
    
    for i in J, j in J
        if i != j 
            @constraint(modelo, Z[i]+p[j]-Z[j] <= M*(1-y[i,j]))
        end
    end
    for i in J
        @constraint(modelo, p[i]<=Z[i])
        @constraint(modelo, Z[i]<= M )
    end
    return modelo
end

creaModeloOrden1 (generic function with 1 method)

In [4]:
# Probar en instancia 1
p, w, A = instancia(1)
m1=creaModeloRect1(p,w,A)
m2=creaModeloOrden1(p,w,A)

A JuMP Model
Minimization problem with:
Variables: 12
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 6 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 14 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 9 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 12 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: Z, y

In [5]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 124 rows, 27 columns and 263 nonzeros
Model fingerprint: 0x931cbb15
Variable types: 0 continuous, 27 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 0 continuous, 27 integer (27 binary)

Root relaxation: objective 2.750000e+01, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   27.50000    0    6          -   27.50000      -     -    0s
     0     0   27.50000    0    6          -   27.50000      -     -    0s
     0     2   28.00000    0    6          -   28.00000      -     -    0s
*    9     5               7      43.0000000   28.50000  33.7%   1.6

In [6]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 20 rows, 12 columns and 39 nonzeros
Model fingerprint: 0x71d4823b
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Variable types: 0 continuous, 12 integer (9 binary)

Root relaxation: objective 2.300000e+01, 9 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   23.00000    0    6          -   23.00000      -     -    0s
     0     0   23.00000    0    6          -   23.00000      -     -    0s
     0     2   23.00000    0    6          -   23.00000      -     -    0s
*    3     2               3      43.0000000   34.00000  20.9%   1.7   

In [7]:
# Probar en instancia 2
p, w, A = instancia(2)
m1_2=creaModeloRect1(p,w,A)
m2_2=creaModeloOrden1(p,w,A)

A JuMP Model
Minimization problem with:
Variables: 12
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 7 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 14 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 9 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 12 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: Z, y

In [8]:
optimize!(m1_2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 133 rows, 27 columns and 353 nonzeros
Model fingerprint: 0x724c2bb2
Variable types: 0 continuous, 27 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 0 continuous, 27 integer (27 binary)

Root relaxation: objective 3.250000e+01, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      43.0000000   43.00000  0.00%     -    0s

Explored 0 nodes (17 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 43 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.300000000000e+01, best bound

In [9]:
optimize!(m2_2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 21 rows, 12 columns and 40 nonzeros
Model fingerprint: 0x66fb1702
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Variable types: 0 continuous, 12 integer (9 binary)

Root relaxation: objective 3.800000e+01, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   38.00000    0    4          -   38.00000      -     -    0s
     0     0   38.00000    0    4          -   38.00000      -     -    0s
     0     2   38.00000    0    4          -   38.00000      -     -    0s
*    1     1               1      70.0000000   41.00000  41.4%   3.0   

In [10]:
# Probar en instancia 3
p, w, A = instancia(3)
m1_3=creaModeloRect1(p,w,A)
m2_3=creaModeloOrden1(p,w,A)

A JuMP Model
Minimization problem with:
Variables: 240
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 120 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1150 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 225 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 225 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 240 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: Z, y

In [11]:
optimize!(m1_3)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 257420 rows, 2130 columns and 516813 nonzeros
Model fingerprint: 0xb07ed08c
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 0 continuous, 2130 integer (2130 binary)

Root relaxation: objective 7.031500e+03, 30 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7031.50000    0   30          - 7031.50000      -     -    0s
     0     0 7031.50000    0   30          - 7031.50000      -     -    0s
     0     2 7034.00000    0   30          - 7034.00000      -     -    1s
    78    80 7928.50000   78   30          - 7043.000

In [12]:
optimize!(m2_3)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 1270 rows, 240 columns and 3615 nonzeros
Model fingerprint: 0x6679c3dd
Variable types: 0 continuous, 240 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Variable types: 0 continuous, 240 integer (225 binary)

Root relaxation: objective 6.642000e+03, 225 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6642.00000    0  210          - 6642.00000      -     -    0s
     0     0 6642.00000    0  210          - 6642.00000      -     -    0s
     0     2 6642.00000    0  210          - 6642.00000      -     -    0s
*  131   131              88    45645.000000 7077.00000  84.

In [13]:
# Probar en instancia 4
p, w, A = instancia(4)
m1_4=creaModeloRect1(p,w,A)
m2_4=creaModeloOrden1(p,w,A)

A JuMP Model
Minimization problem with:
Variables: 240
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 124 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1150 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 225 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 225 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 240 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: Z, y

In [14]:
optimize!(m1_4)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 257988 rows, 2130 columns and 598037 nonzeros
Model fingerprint: 0xacece78e
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 0 continuous, 2130 integer (2130 binary)

Root relaxation: objective 7.299500e+03, 30 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9760.50000    0   30          - 9760.50000      -     -    1s
     0     0 9894.50000    0   30          - 9894.50000      -     -    2s
     0     2 9900.50000    0   30          - 9900.50000      -     -    2s
    35    29 12567.0000   21   24          - 9962.000

In [15]:
optimize!(m2_4)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 1274 rows, 240 columns and 3619 nonzeros
Model fingerprint: 0x6111f06b
Variable types: 0 continuous, 240 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Variable types: 0 continuous, 240 integer (225 binary)

Root relaxation: objective 9.509000e+03, 203 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9509.00000    0  156          - 9509.00000      -     -    0s
     0     0 9509.00000    0  156          - 9509.00000      -     -    0s
     0     2 9509.00000    0  156          - 9509.00000      -     -    0s
*  109   109              73    48368.000000 11030.0000  77.

## 🔴 Ejercicio 2. Transformación de soluciones


2.a)
Escriba una función ```generafactible(orden, p, modelo)``` que recibe:
* Un arreglo ```orden``` que contiene una permutación del arreglo 1:N (donde N es el largo de p)
* Un arreglo ```p``` de pesos de una instancia
* Un índice ```modelo``` en {1,2}

Si ```modelo=1``` debe devolver la solución asociada al orden entregado  para Rect1 y si ```modelo=2```
debe devolver la solución asociada al orden para Rect2.

Por ejemplo, la solución asociada a p=[2,4,3] y orden=[3,2,1] debe tener tiempos de completación C(3)=3, C(2)=3+4=7, C(1)=3+4+2=9. Con esto, si ejecuta:

(Ejecute estas lineas al final de 2.a)
```julia
orden=[3,2,1];
p=[2,4,3];
QQ=generafactible(orden,p,1);
ZZ,yy=generafactible(orden,p,2);
@show QQ;
@show ZZ;
@show yy;
```
debe obtener como salida
```julia
QQ = [0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 1 0 0; 0 0 1 0 0 0 0 0 0]
ZZ = [9, 7, 3]
yy = [1 0 0; 1 1 0; 1 1 1]
```


2.b) 

En la parte anterior se solicitó una función que dado un orden entrega soluciones factibles para los modelos creados. En esta parte debe hacer el proceso inverso: escribir una función que recupere el orden de los trabajos a partir de una solución factible.

Más precisamente, escriba dos funciones ```generapermutacionRect(QQ)``` y ```generapermutacionOrden(ZZ,yy)``` que hagan lo siguiente

* Dado una matriz QQ factible para Rect1, ```generapermutacionRect(QQ)``` entrega el orden en el cual están asignados los trabajos en esta solución.

* Dado (ZZ, yy) factible para Orden1, ```generapermutacionOrden(ZZ,yy)``` entrega el orden en el cual están asignados los trabajos en esta solución.

Siguiendo el ejemplo anterior, si ahora ejecuta.

(Ejecute estas lineas al final de 2.b)
```julia
QQ = [0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 1 0 0; 0 0 1 0 0 0 0 0 0];
ZZ = [9, 7, 3];
yy = [1 0 0; 1 1 0; 1 1 1];
orden1=generapermutacionRect(QQ);
orden2=generapermutacionOrden(ZZ,yy);
@show orden1
@show orden2
```
debe obtener como salida
```
orden1 = [3, 2, 1]
orden2 = [3, 2, 1]
```


In [16]:
# 2.a)
function generafactible(orden, p, modelo)
    
C = zeros( Int, ( length(p),1))
    a=0
    for k in orden
        b = p[k]+ a
        a = b
        C[k]= b
    end

    
    if modelo==1
        N = sum(p[i] for i in 1:length(p))
        QQ= zeros(Int, ( length(p),N))
        for i in orden
            QQ[i, C[i]]=1
        end
        return QQ
    elseif modelo==2
        n=length(orden)
        ZZ=Int[]
        for i in 1:n
            append!(ZZ,C[i,:])
        end
        yy= zeros(Int, (length(p), length(p)))
        for i in 1:length(orden)
            for j in 1:length(orden)
                if orden[i] <= orden[j]
                    yy[i,j] =1
                end
            end
        end
        return ZZ,yy
    end
    @error("modelo debe ser 1 o 2")
end

generafactible (generic function with 1 method)

In [17]:
# prueba 2.a
orden=[3,2,1];
p=[2,4,3];
QQ=generafactible(orden,p,1);
ZZ,yy=generafactible(orden,p,2);
@show QQ;
@show ZZ;
@show yy;

QQ = [0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 1 0 0; 0 0 1 0 0 0 0 0 0]
ZZ = [9, 7, 3]
yy = [1 0 0; 1 1 0; 1 1 1]


In [18]:
# 2.b)
function generapermutacionRect(QQ)
    # CUIDADO! QQ podria tener valores muy cercanos a 1, pero no ser iguales a 1 por problemas de precisión
    # Si necesita revisar que QQ[i,t] es 1, use QQ[i,t]≈1
    
    orden=Int[]
    QQT=transpose(QQ)
    for f in 1:(size(QQT)[1])
        for c in 1:(size(QQT)[2])
            if QQT[f,c] ≈ 1
                append!(orden,c)
            end
        end
    end
    return orden
end

function generapermutacionOrden(ZZ,yy)
    n=length(yy[1,:])
    orden=zeros(Int,n)
    for f in 1:(size(yy)[1])
        suma=sum(yy[f,c] for c in 1:(size(yy)[2]))
        orden[n-suma+1]= f
    end
    return orden
end

generapermutacionOrden (generic function with 1 method)

In [19]:
# prueba 2.b
QQ = [0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 1 0 0; 0 0 1 0 0 0 0 0 0];
ZZ = [9, 7, 3];
yy = [1 0 0; 1 1 0; 1 1 1];
orden1=generapermutacionRect(QQ);
orden2=generapermutacionOrden(ZZ,yy);
@show orden1
@show orden2

orden1 = [3, 2, 1]
orden2 = [3, 2, 1]


3-element Array{Int64,1}:
 3
 2
 1

## 🔴 Ejercicio 3. Generando buenas soluciones iniciales para ambos modelos.

3.a)
Sabemos que si no hay restricciones de precedencias, el orden dado por mayor a menor eficiencia es una solución óptima para el problema. Este orden se puede obtener en julia a partir de w y p escribiendo
```julia
eficiencia = sort(1:length(p), by=x->w[x]/p[x], rev=true)
```
Es posible que este orden no sea compatible con las restricciones de precedencias dadas por el digrafo (J,A) en ciertas instancias por lo que no puede ser usado como solucion inicial directamente.

Una alternativa simple para obtener un orden inicial ```inicial``` bueno es la siguiente.

Para cada elemento i, determine el conjunto prec[i] de sus predecesores, es decir el conjunto de todos los elementos j tal que [i,j] está en A. Partiendo del arreglo ```inicial``` vacío y hasta que este arreglo contenga todos los elementos de J, determine iterativamente cual es el primer elemento del arreglo ```eficiencia``` tal que todos sus predecesores están en ```inicial```, una vez encontrado dicho elemento, agréguelo al final del arreglo  ```inicial``` y elimínelo de  ```eficiencia```.

Haga todo lo anterior dentro de una función  ```ordeneficiente(p,w,A)``` que entregue el arreglo ```inicial``` descrito arriba. Luego, pruebe su función con los arreglos de la instancia 4, en particular si escribe
```julia
p,w,A=instancia(4)
inicial=ordeneficiente(p,w,A)
@show inicial
```
debería obtener
```julia
inicial = [3, 15, 10, 1, 8, 12, 13, 14, 2, 11, 9, 6, 5, 7, 4]
```

3.b)
Usando las funciones creadas en 3.a y 2.a. 

Cree y optimice ambos modelos en las instancias i=1, i=2, i=3, i=4 pero ahora usando como solución inicial la solución factible asociada al orden obtenido mediante ```orden_eficiente(p,w,A)```. Llamamos a esto solucion inicial eficiente

3.c) Pregunta abierta: ¿Cómo compara (gap, calidad, tiempo, etc.) las soluciones encontradas en 2.c, con las soluciones encontradas en el ejercicio 1?


In [20]:
# 3.a)
# Hint. algunos comandos utiles para manejar arreglos son push!, deleteat!, 

function ordeneficiente(p, w, A)
    eficiencia = sort(1:length(p), by=x->w[x]/p[x], rev=true)
    N=length(p)
    prec=[[] for i=1:N]
    
    # Para todo i, llenar en prec[i] el conjunto de todos los predecesores de i
    for i in 1:N
        for j in 1:N
            if [j,i] in A
                push!(prec[i],j)
            end
        end  
    end
    
    # construir arreglo inicial
    inicial=Int[]
    while(!isempty(eficiencia))
         # encontrar primer indice i tal que todas las precedencias de eficiencia[i] están en inicial


        j=0
        for i in 1:N
            Conj=prec[eficiencia[i]]
            if issubset(Conj,inicial)
                j=i
                break
            end
        end    
        # al encontrarlo, agregar eficiencia[i] al final de inicial y eliminarlo  de eficiencia
            push!(inicial,eficiencia[j])
 
            deleteat!(eficiencia,j)

        
    end

    return inicial
    
end

ordeneficiente (generic function with 1 method)

In [21]:
# 3.a) prueba
p,w,A=instancia(4)
inicial=ordeneficiente(p,w,A)
@show inicial;

inicial = [3, 15, 10, 1, 8, 12, 13, 14, 2, 11, 9, 6, 5, 7, 4]


In [22]:
# 3.b) Probar ambos modelos en instancia 1

p, w, A = instancia(1)
inicial=ordeneficiente(p,w,A)

# crear modelos (Rect1) y modelo2 (Orden1)
m1=creaModeloRect1(p,w,A)
m2=creaModeloOrden1(p,w,A)
# obtener solucion inicial QQ a partir de inicial
QQ=generafactible(inicial, p, 1)
ZZ,yy=generafactible(inicial, p, 2)
# fijar QQ como solucion inicial de m1
# fijar (ZZ,yy) como solucion inicial de m2
for i in 1:length(QQ)
    set_start_value.(m1[:Q][i],QQ[i])
end
for i in 1:length(ZZ)
    for j in 1:length(ZZ)
        set_start_value.(m2[:y][i,j],yy[i,j])
    end
end
for i in 1:length(ZZ)
    set_start_value.(m2[:Z][i],ZZ[i])
end

In [23]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 124 rows, 27 columns and 263 nonzeros
Model fingerprint: 0x0dd53c49
Variable types: 0 continuous, 27 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective 34

Variable types: 0 continuous, 27 integer (27 binary)

Root relaxation: objective 2.750000e+01, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        34.00000   34.00000  0.00%     -    0s

Explored 0 nodes (6 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 34 

Optimal solution found (tolerance 1.00e-04)
Best

In [24]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 20 rows, 12 columns and 39 nonzeros
Model fingerprint: 0xd5dac56b
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]

Loaded user MIP start with objective 34

Variable types: 0 continuous, 12 integer (9 binary)

Root relaxation: objective 2.300000e+01, 9 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   23.00000    0    6   34.00000   23.00000  32.4%     -    0s
     0     0     cutoff    0        34.00000   34.00000  0.00%     -    0s

Explored 1 nodes (11 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

In [25]:
# 3.b) Probar ambos modelos en instancia 2
p, w, A = instancia(2)
inicial=ordeneficiente(p,w,A)

# crear modelos (Rect1) y modelo2 (Orden1)
m1=creaModeloRect1(p,w,A)
m2=creaModeloOrden1(p,w,A)
# obtener solucion inicial QQ a partir de inicial
QQ=generafactible(inicial, p, 1)
ZZ,yy=generafactible(inicial, p, 2)
# fijar QQ como solucion inicial de m1
# fijar (ZZ,yy) como solucion inicial de m2
for i in 1:length(QQ)
    set_start_value.(m1[:Q][i],QQ[i])
end
for i in 1:length(ZZ)
    for j in 1:length(ZZ)
        set_start_value.(m2[:y][i,j],yy[i,j])
    end
end
for i in 1:length(ZZ)
    set_start_value.(m2[:Z][i],ZZ[i])
end

In [26]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 133 rows, 27 columns and 353 nonzeros
Model fingerprint: 0x116d3f02
Variable types: 0 continuous, 27 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective 43

Variable types: 0 continuous, 27 integer (27 binary)

Root relaxation: objective 3.250000e+01, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        43.00000   43.00000  0.00%     -    0s

Explored 0 nodes (7 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 43 

Optimal solution found (tolerance 1.00e-04)
Best

In [27]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 21 rows, 12 columns and 40 nonzeros
Model fingerprint: 0x29db83ff
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R11 by 6.000000000

Variable types: 0 continuous, 12 integer (9 binary)

Root relaxation: objective 3.800000e+01, 8 iterations, 0.00 seconds
Another try with MIP start

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   38.00000    0    4          -   38.00000      -     -    0s
     0     0   38.00000    0    4          -   38.00000      -     -    0s
     0 

In [28]:
# 3.b) Probar ambos modelos en instancia 3
p, w, A = instancia(3)
inicial=ordeneficiente(p,w,A)

# crear modelos (Rect1) y modelo2 (Orden1)
m1=creaModeloRect1(p,w,A)
m2=creaModeloOrden1(p,w,A)
# obtener solucion inicial QQ a partir de inicial
QQ=generafactible(inicial, p, 1)
ZZ,yy=generafactible(inicial, p, 2)
# fijar QQ como solucion inicial de m1
# fijar (ZZ,yy) como solucion inicial de m2
for i in 1:length(QQ)
    set_start_value.(m1[:Q][i],QQ[i])
end
for i in 1:length(ZZ)
    for j in 1:length(ZZ)
        set_start_value.(m2[:y][i,j],yy[i,j])
    end
end
for i in 1:length(ZZ)
    set_start_value.(m2[:Z][i],ZZ[i])
end

In [29]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 257420 rows, 2130 columns and 516813 nonzeros
Model fingerprint: 0x04e03863
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective 35578

Variable types: 0 continuous, 2130 integer (2130 binary)

Root relaxation: objective 7.031500e+03, 30 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7031.50000    0   30 35578.0000 7031.50000  80.2%     -    0s
     0     0 7031.50000    0   30 35578.0000 7031.50000  80.2%     -    0s
     0     2 7034.00000    0   30 35578.0000 7034.00000  80.2%     -    1s
    50  

In [30]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 1270 rows, 240 columns and 3615 nonzeros
Model fingerprint: 0xc8b149ce
Variable types: 0 continuous, 240 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R1045 by 84.000000000

Variable types: 0 continuous, 240 integer (225 binary)

Root relaxation: objective 6.642000e+03, 225 iterations, 0.00 seconds
Another try with MIP start

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                    47706.000000          -      -     -    0s
     0     0 6642.00000    0  210 47706.0000 6642.00000  86.1%     -

In [31]:
# 3.b) Probar ambos modelos en instancia 4
p, w, A = instancia(4)
inicial=ordeneficiente(p,w,A)

# crear modelos (Rect1) y modelo2 (Orden1)
m1=creaModeloRect1(p,w,A)
m2=creaModeloOrden1(p,w,A)
# obtener solucion inicial QQ a partir de inicial
QQ=generafactible(inicial, p, 1)
ZZ,yy=generafactible(inicial, p, 2)
# fijar QQ como solucion inicial de m1
# fijar (ZZ,yy) como solucion inicial de m2
for i in 1:length(QQ)
    set_start_value.(m1[:Q][i],QQ[i])
end
for i in 1:length(ZZ)
    for j in 1:length(ZZ)
        set_start_value.(m2[:y][i,j],yy[i,j])
    end
end
for i in 1:length(ZZ)
    set_start_value.(m2[:Z][i],ZZ[i])
end

In [32]:
optimize!(m1)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 257988 rows, 2130 columns and 598037 nonzeros
Model fingerprint: 0x58a86bfb
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective 41527

Variable types: 0 continuous, 2130 integer (2130 binary)

Root relaxation: objective 7.299500e+03, 30 iterations, 0.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9760.50000    0   30 41527.0000 9760.50000  76.5%     -    1s
     0     0 9894.50000    0   30 41527.0000 9894.50000  76.2%     -    2s
     0     2 9900.50000    0   30 41527.0000 9900.50000  76.2%     -    2s
    37  

In [33]:
optimize!(m2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 1274 rows, 240 columns and 3619 nonzeros
Model fingerprint: 0x821bf92b
Variable types: 0 continuous, 240 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R120 by 1.000000000

Variable types: 0 continuous, 240 integer (225 binary)

Root relaxation: objective 9.509000e+03, 203 iterations, 0.00 seconds
Another try with MIP start

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9509.00000    0  156          - 9509.00000      -     -    0s
     0     0 9509.00000    0  156          - 9509.00000      -     -  

 🔴 3.c)
 
 ¿Cómo compara (tiempo, calidad, etc.) las soluciones encontradas en 3.b, con las soluciones encontradas en el ejercicio 1?
 
 Escribir respuesta aqui:
 
 Se observa que en los modelos hechos en el ejercicio 1 toman más iteraciones de simplex que en los de el ejercicio 3.b. Sin embargo, los valores objetivos encontrados coinciden junto cno otros parámetros de precisión.
 

## 🔴 Ejercicio 4. Mejorando Rect1 con CliqueCuts

4.a) 

Nuevamente cree un modelo Rect1 para las instancias i=3, i=4, al igual que la parte anterior
use la solucion inicial eficiente. Antes de optimizar, agregue generación agresiva de CliqueCuts y resuelva.

Luego responda la pregunta abierta: ¿Cómo compara las soluciones encontradas en 4.a, con las soluciones encontradas en 3.b?

4.b)

De acuerdo a lo explicado en la sección D.3, podemos obtener un mejor modelo para nuestro problema generando nosotros mismos los CliqueCuts. Esto ese logra reemplazando en el modelo (rect1) las restricciones

$$Q_{i,s}+Q_{j,t} \leq 1,\quad \forall i< j, \forall s,t\in [T]: (t-p_j+1\leq s) \wedge (s-p_i+1\leq t)$$

por las restricciones

$$\sum_{(i,s)\in I(t)} Q_{i,s} \leq 1, \quad \forall t\in T$$
donde $I(t) = \{(i,s)\colon i\in J, s\in [T], s-p_i+1\leq t\leq s\}$

Llamemos Rect2 al modelo resultante.

En base a esto, escriba una función ```creaModeloRect2(p,w,A)``` que retorne un modelo tipo Rect2 para el problema, usando las mismas opciones que en la pregunta 1.a)

4.c) 
Use este modelo para resolver las instancias i=3, i=4 (no genere CliqueCuts) y use solución inicial eficiente. 
Pregunta abierta: ¿Cómo compara las soluciones (tiempo, calidad, etc.) encontradas en 4.c con las de 4.a?

In [34]:
# 4.a) i=3. Cliquecuts en Rect1
p, w, A = instancia(3)
inicial=ordeneficiente(p,w,A)
m=creaModeloRect1(p,w,A)
QQ=generafactible(inicial, p, 1)
# fijar QQ como solucion inicial de m
set_start_value.(m[:Q],QQ)

# agregar CliqueCuts
set_optimizer_attribute(m,"CliqueCuts",2)
# optimizar
optimize!(m)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 257420 rows, 2130 columns and 516813 nonzeros
Model fingerprint: 0x04e03863
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective 35578

Variable types: 0 continuous, 2130 integer (2130 binary)

Root relaxation: objective 7.031500e+03, 30 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7031.50000    0   30 35578.0000 7031.50000  80.2%     -    0s
     0     0 9116.00000    0   30 35578.0000 9116.00000  74.4%     -    0s
     0     0 10902.5000    0   30 35578.0000 10902.5000  69.4%     -    1s
     0  

In [35]:
# 4.a) i=4. Cliquecuts en Rect1
p, w, A = instancia(4)
inicial=ordeneficiente(p,w,A)
m=creaModeloRect1(p,w,A)
QQ=generafactible(inicial, p, 1)
# fijar QQ como solucion inicial de m
set_start_value.(m[:Q],QQ)

# agregar CliqueCuts
set_optimizer_attribute(m,"CliqueCuts",2)
# optimizar
optimize!(m)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 257988 rows, 2130 columns and 598037 nonzeros
Model fingerprint: 0x58a86bfb
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective 41527

Variable types: 0 continuous, 2130 integer (2130 binary)

Root relaxation: objective 7.299500e+03, 30 iterations, 0.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9760.50000    0   30 41527.0000 9760.50000  76.5%     -    1s
     0     0 18208.0000    0   30 41527.0000 18208.0000  56.2%     -    2s
     0     0 20101.8295    0   43 41527.0000 20101.8295  51.6%     -    2s
     0  

4.a) 🔴 Pregunta abierta. 

¿Cómo compara las soluciones encontradas en 4.a, con las soluciones encontradas en 3.b?

Implementando CliqueCuts, el modelo 1 es capaz de hacer más iteraciones en el mismo tiempo, es decir, la transición de iteraciones es más rápida, por lo que cliqueCuts es una mejora en cuanto a encontrar el valor objetivo se refiere.


In [36]:
# 4.b)
function creaModeloRect2(p,w,A)
    n =  length(p)
    J = [i for i in 1:n]
    T = sum(p[i] for i in J)
    modelo = Model(optimizer_with_attributes(
        () -> Gurobi.Optimizer(GUROBI_ENV),
        "TimeLimit" => 15,
        "Threads" => 1,
        "Presolve" => 0,
        "Heuristics" => 0,
        "Cuts" => 0))
    # Crear variables, objetivos y restricciones
    @variable(modelo, 0<=Q[i in 1:n,t in 1:T]<=1, Int)   
    
    @objective(modelo, Min, sum( sum( Q[i ,t]*t*w[i] for t in 1:T) for i in J ))
    for i in J
    @constraint(modelo, sum( Q[i, t] for t in 1:T) == 1 )
        for t in 1:T
            if t<= p[i]- 1
            @constraint(modelo, Q[i,t] == 0)
            end
        end
    end
    
    for t in 1:T
    I= [[i,s] for i in J, s in 1:T if (s-p[i]+1 <= t && t <= s)]
    @constraint(modelo, sum(Q[e[1],e[2]] for e in I)<=1)
    end
    
    for s in 1:T, e in A
        @constraint(modelo, sum(Q[e[1], t ] for t in 1:s) >= sum(Q[e[2], t ] for t in 1:s))
        end
    return modelo
end

creaModeloRect2 (generic function with 1 method)

In [37]:
# 4.c) i=3 usar modelo Rect2, con solucion inicial.
p, w, A = instancia(3)
inicial=ordeneficiente(p,w,A)
m=creaModeloRect2(p,w,A) #modelo Rect2
QQ=generafactible(inicial, p, 1)
# fijar QQ como solucion inicial de m 
set_start_value.(m[:Q],QQ)

optimize!(m)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 284 rows, 2130 columns and 21699 nonzeros
Model fingerprint: 0x88f622f3
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective 35578

Variable types: 0 continuous, 2130 integer (2130 binary)

Root relaxation: cutoff, 647 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      35578.0000 35578.0000  0.00%     -    0s

Explored 0 nodes (647 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 35578 

Optimal solution found (tolerance 1.00e-04)


In [38]:
# 4.c) i=4 usar modelo Rect2, con solucion inicial.
p, w, A = instancia(4)
inicial=ordeneficiente(p,w,A)
m=creaModeloRect2(p,w,A) #modelo Rect2
QQ=generafactible(inicial, p, 1)
# fijar QQ como solucion inicial de m 
set_start_value.(m[:Q],QQ)

optimize!(m)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 852 rows, 2130 columns and 102923 nonzeros
Model fingerprint: 0x740d2387
Variable types: 0 continuous, 2130 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective 41527

Variable types: 0 continuous, 2130 integer (2130 binary)

Root relaxation: objective 3.748232e+04, 964 iterations, 0.07 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 37487.5913    0   55 41527.0000 37487.5913  9.73%     -    0s
     0     0 37487.5913    0   55 41527.0000 37487.5913  9.73%     -    0s
     0     2 37487.5913    0   55 41527.0000 37487.5913  9.73%     -    0s
*    9    

4.c) Pregunta abierta: ¿Cómo compara las soluciones (tiempo, calidad, etc.) encontradas en 4.c con las de 4.a?

Comparando estos modelos, observamos que en el modelo rect2 la disminución de tiempo fue significativa con respecto al modelo anterior por lo que es una mejora el cambiar la restriccion de big-M. Por otro lado, hay una pequeña diferencia en el valor objetivo encontrado pues el primer modelo alcanza el límite de tiempo exigido.

## 🔴 Ejercicio 5. Eliminando big-M en modelo Orden1


El problema central del modelo Orden1 es la existencia de restricciones de tipo big-M.

$$Z_i+p_j-Z_j\leq M(1-y_{i,j}), \forall i, j\in J, i\neq j$$

Una forma interesante de cambiar estas restricciones es notar que el tiempo de completación se puede calcular directamente a partir de las variables $y$ como sigue

$$Z_i = \sum_{j\in J}y_{j,i}p_j$$

Con esto automáticamente sabemos que si $y_{i,j}$ es 1 entonces $Z_i+p_i\leq Z_j$.

Llamemos entonces Orden2 al modelo obtenido reemplazando las restricciones
$$Z_i+p_j-Z_j\leq M(1-y_{i,j}), \forall i, j\in J, i\neq j$$
por las restricciones
$$Z_i = \sum_{j\in J}y_{j,i}p_j, \forall i\in J$$


5.a) 
    Escriba una función ```creaModeloOrden2(p,w,A)``` que retorne un modelo tipo Orden2 para el problema (use las mismas opciones que en la pregunta 1.a) 

5.b) 
    Use este modelo para resolver las instancias i=3, i=4 partiendo de la solución inicial eficiente.

5.c) Pregunta abierta: ¿Cómo compara las soluciones (tiempo, calidad, etc.) asociadas al modelo Orden2 con las del modelo Orden1 resueltas hasta ahora?


In [39]:
# 5.a) 
function creaModeloOrden2(p,w,A)
    n =  length(p)
    J = [i for i in 1:n]
    M = sum(p[i] for i in J)
    modelo = Model(optimizer_with_attributes(
        () -> Gurobi.Optimizer(GUROBI_ENV),
        "TimeLimit" => 15,
        "Threads" => 1,
        "Presolve" => 0,
        "Heuristics" => 0,
        "Cuts" => 0))
    # Crear variables, objetivos y restricciones
    @variable(modelo, 0<=y[i in J,j in J]<=1, Int)
    
    @variable(modelo, Z[i in J], Int)
    
    @objective(modelo, Min, sum(Z[i]*w[i] for i in J ))
    for i in J 
        @constraint(modelo, y[i,i] == 1)
        for j in J
            if i< j
                @constraint(modelo, y[i,j]+y[j,i] ==1)
            end
            for k in J
                if (i<j && j<k)|| (i<k && k<j )
                    @constraint(modelo, y[i,j]+y[j,k]+y[k,i] <=2)
                end
            end
            
        end
    end
    for e in A
        @constraint(modelo,y[e[1], e[2]] ==1)
    end
  ########## restricción a cambiar  
    #for i in J, j in J
    #    if i != j 
    #        @constraint(modelo, Z[i]+p[j]-Z[j] <= M*(1-y[i,j]))
    #    end
    #end
    for i in J
        @constraint(modelo, sum( y[j, i]*p[j] for j in J) == Z[i] )
    end
  ##########
    for i in J
        @constraint(modelo, p[i]<=Z[i])
        @constraint(modelo, Z[i]<= M )
    end
    return modelo
end

creaModeloOrden2 (generic function with 1 method)

In [40]:
# 5.b) i=3 usar modelo Orden2, con solucion inicial.
p, w, A = instancia(3)
inicial=ordeneficiente(p,w,A)
m=creaModeloOrden2(p,w,A) #modelo Orden2
ZZ,yy=generafactible(inicial, p, 2)
for i in 1:length(ZZ)
    for j in 1:length(ZZ)
        set_start_value.(m2[:y][i,j],yy[i,j])
    end
end
for i in 1:length(ZZ)
    set_start_value.(m2[:Z][i],ZZ[i])
end
optimize!(m)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 1075 rows, 240 columns and 3225 nonzeros
Model fingerprint: 0x6d212125
Variable types: 0 continuous, 240 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Variable types: 0 continuous, 240 integer (225 binary)

Root relaxation: objective 3.557800e+04, 105 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    35578.000000 35578.0000  0.00%     -    0s

Explored 0 nodes (105 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 35578 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.557800000000e+04

In [41]:
# 5.b) i=4 usar modelo Orden2, con solucion inicial.
p, w, A = instancia(4)
inicial=ordeneficiente(p,w,A)
m=creaModeloOrden2(p,w,A) #modelo Orden2
ZZ,yy=generafactible(inicial, p, 2)
for i in 1:length(ZZ)
    for j in 1:length(ZZ)
        set_start_value.(m2[:y][i,j],yy[i,j])
    end
end
for i in 1:length(ZZ)
    set_start_value.(m2[:Z][i],ZZ[i])
end
optimize!(m)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 1079 rows, 240 columns and 3229 nonzeros
Model fingerprint: 0xf09b7ab0
Variable types: 0 continuous, 240 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Variable types: 0 continuous, 240 integer (225 binary)

Root relaxation: objective 3.801700e+04, 108 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    38017.000000 38017.0000  0.00%     -    0s

Explored 0 nodes (108 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 38017 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.801700000000e+04

5.c) Pregunta abierta: ¿Cómo compara las soluciones (tiempo, calidad, etc.) asociadas al modelo Orden2 con las del modelo Orden1 resueltas hasta ahora (3b)?

Las soluciones asociadas al modelo orden 2 se encuentran de manera más rapida. En el modelo orden 1 se realizan más iteraciones, dado que se encuentran más soluciones factibles. El valor objetivo del modelo orden 2 es más chico que el encontrado en el orden 1.  

## 🔴 Ejercicio 6. Analisis completo


En este problema deberá comparar las 4 soluciones siguientes 


* Modelo1: Rect1, *con* CliqueCuts, con solución inicial eficiente.
* Modelo2: Rect2,  con solución inicial eficiente.
* Modelo3: Orden1, con solución inicial eficiente.
* Modelo4: Orden2, con solución inicial eficiente.

en las instancias 1 a 8 provistas, con el mismo tiempo límite (15 segundos).

Más precisamente cree una función

```Julia
function Benchmark()
return df
```

donde df es un `dataframe` cuyas filas están indexadas por instancia, y las columnas son
    
| tiempo1 | gap1 | valor1 | tiempo2 | gap2 | valor2 | tiempo3 | gap3 | valor3 | tiempo4 | gap4 | valor4 | ganador | mejororden |

con
tiempo j = tiempo usado en la resolución de la instancia, usando formulacion Modeloj, 
           usando MOI.get(modelo, MOI.SolveTime())
           
gap j    = gap de resolución en porcentaje (0% = optimo, infinito si no encontró solución factible)  
           usando MOI.get(modelo, MOI.RelativeGap())
           
valor j  = valor objetivo encontrado  

ganador   = mejor modelo de los 4 para esta instancia. Esto se calcula de la siguiente manera:  
           1) Revisar si algun modelo encontró el óptimo revisando si  
              termination_status(model) == MOI.OPTIMAL  
           2) Si uno o mas encontraron el optimo, el mejor es aquel con menor tiempo de resolución.  
           3) Si ninguno encuentra el optimo, aquel con menor gap.  

mejororden = orden de los trabajos para el modelo ganador en esta instancia (encontrado usando las funciones de las partes anteriores).  

Al finalizar, ejecute la función e imprima en pantalla el dataframe con
```julia
df=Benchmark()
ENV["COLUMNS"]=1000
df
```

**Advertencia:** La ejecución de la función podría tomar hasta 10 minutos. Planee con tiempo.

In [42]:
function Ganador(M,T,G) #M=[m1,m2,m3,m4] y T,G son listas de tiempos y gaps
    ts1=termination_status(M[1])
    ts2=termination_status(M[2])
    ts3=termination_status(M[3])
    ts4=termination_status(M[4])
    TS=[ts1,ts2,ts3,ts4]
    t=[100.0,100.0,100.0,100.0]
    if (ts1,ts2,ts3,ts4)==(MOI.OPTIMAL, MOI.OPTIMAL, MOI.OPTIMAL, MOI.OPTIMAL)
        ganador=M[findmin(T)[2]]
    elseif ts1!=MOI.OPTIMAL && ts2!=MOI.OPTIMAL && ts3!=MOI.OPTIMAL && ts4!=MOI.OPTIMAL
        ganador=M[findmin(G)[2]]
    else
        for i in 1:4
            if TS[i]==MOI.OPTIMAL
                t[i]=T[i]
            end
        end
        tiempoganador=findmin(t)[1]
        for i in 1:4
            if tiempoganador==T[i]
                ganador=M[i]
            end
        end
        end
    return ganador
end

Ganador (generic function with 1 method)

In [44]:
# 6) 
function Benchmark()
    df = DataFrame(
        tiempo1 = Float64[],
        gap1 = Float64[],
        valor1 = Float64[],
        tiempo2 = Float64[],
        gap2 = Float64[],
        valor2 = Float64[],
        tiempo3 = Float64[],
        gap3 = Float64[],
        valor3 = Float64[],
        tiempo4 = Float64[],
        gap4 = Float64[],
        valor4 = Float64[],
        ganador = Any[],
        mejororden = Any[])
    
    
    for i in 1:8
        # Crear instancia y solucion inicial
        p,w,A=instancia(i)
        inicial=ordeneficiente(p,w,A)
        QQ=generafactible(inicial, p, 1)
        ZZ,yy=generafactible(inicial,p,2)
        
        m1=creaModeloRect1(p,w,A) #MODELO 1
        set_start_value.(m1[:Q],QQ)
        set_optimizer_attribute(m1,"CliqueCuts",2)
    
        m2=creaModeloRect2(p,w,A) #MODELO 2
        set_start_value.(m2[:Q],QQ)
    
        m3=creaModeloOrden1(p,w,A) #MODELO 3
        for i in 1:length(ZZ)
            for j in 1:length(ZZ)
                set_start_value.(m3[:y][i,j],yy[i,j])
            end
        end
        for i in 1:length(ZZ)
            set_start_value.(m3[:Z][i],ZZ[i])
        end
    
        m4=creaModeloOrden2(p,w,A) #MODELO 4
        for i in 1:length(ZZ)
            for j in 1:length(ZZ)
                set_start_value.(m4[:y][i,j],yy[i,j])
            end
        end
        for i in 1:length(ZZ)
            set_start_value.(m4[:Z][i],ZZ[i])
        end
        # Resolver cada uno y extraer parametros pedidos (puede usar 
        #MOI.get(m1, MOI.SolveTime()), MOI.get(m1, MOI.RelativeGap())
    
        optimize!(m1)
        tiempo1=MOI.get(m1, MOI.SolveTime())
        gap1=MOI.get(m1, MOI.RelativeGap())
        valor1=objective_value(m1)
        optimize!(m2)
        tiempo2=MOI.get(m2, MOI.SolveTime())
        gap2=MOI.get(m2, MOI.RelativeGap())
        valor2=objective_value(m2)
        optimize!(m3)
        tiempo3=MOI.get(m3, MOI.SolveTime())
        gap3=MOI.get(m3, MOI.RelativeGap())
        valor3=objective_value(m3)
        optimize!(m4)
        tiempo4=MOI.get(m4, MOI.SolveTime())
        gap4=MOI.get(m4, MOI.RelativeGap())
        valor4=objective_value(m4)
        # Determinar ganador (de acuerdo a GAP) y mejor ordedn
        ganador=Ganador([m1,m2,m3,m4],[tiempo1,tiempo2,tiempo3,tiempo4],[gap1,gap2,gap3,gap4])
        mejororden = []
        if ganador ==m1 || ganador ==m2
            mejororden=generapermutacionRect(QQ)
        elseif ganador ==m3 || ganador == m4
            mejororden=generapermutacionOrden(ZZ,yy)
        end
        G = " "
        if ganador == m1
            G = "Modelo1"
        elseif ganador == m2
            G = "Modelo2"
        elseif ganador == m3
            G = "Modelo3"
        else
            G = "Modelo4"
        end
        # agregar fila a dataframe
        push!(df, [tiempo1, gap1, valor1,tiempo2,gap2,valor2,tiempo3,gap3,valor3,tiempo4,gap4,valor4, G, mejororden  ])
    end
    
    return df
end

Benchmark (generic function with 1 method)

In [45]:
df = Benchmark()
ENV["COLUMNS"]=1000
df

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 124 rows, 27 columns and 263 nonzeros
Model fingerprint: 0x0dd53c49
Variable types: 0 continuous, 27 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective 34

Variable types: 0 continuous, 27 integer (27 binary)

Root relaxation: objective 2.750000e+01, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        34.00000   34.00000  0.00%     -    0s

Explored 0 nodes (6 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 34 

Optimal solution found (tolerance 1.00e-04)
Best

,tiempo1,gap1,valor1,tiempo2,gap2,valor2,tiempo3,gap3,valor3,tiempo4,gap4,valor4,ganador,mejororden
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Any,Any
1,0.000999451,0.0,34.0,0.0,0.0,34.0,0.000997543,0.0,34.0,0.0,0.0,34.0,Modelo2,"[1, 3, 2]"
2,0.000995636,0.0,43.0,0.000997543,0.0,43.0,0.000997543,0.0,43.0,0.0,0.0,43.0,Modelo4,"[2, 3, 1]"
3,15.0121,0.0659115,35578.0,0.0239391,0.0,35578.0,15.0002,0.642047,36312.0,0.00299263,0.0,35578.0,Modelo4,"[1, 11, 2, 15, 7, 14, 8, 5, 13, 4, 12, 6, 9, 10, 3]"
4,15.0486,0.200303,41527.0,1.40773,0.0,38017.0,15.0004,0.517431,40187.0,0.00196648,0.0,38017.0,Modelo4,"[4, 9, 1, 15, 13, 12, 14, 5, 11, 3, 10, 6, 7, 8, 2]"
5,15.1236,0.425669,43562.0,0.035902,0.0,43562.0,15.0006,0.812281,47699.0,0.00994301,0.0,43562.0,Modelo4,"[20, 22, 15, 4, 25, 19, 10, 16, 1, 5, 23, 12, 2, 7, 13, 21, 17, 3, 11, 9, 8, 18, 24, 6, 14]"
6,15.0292,0.247041,50951.0,15.0012,0.00274083,47066.0,15.0008,0.742352,50930.0,0.00900841,0.0,47045.0,Modelo4,"[16, 20, 11, 3, 25, 14, 12, 13, 15, 4, 23, 9, 1, 5, 10, 19, 22, 2, 7, 6, 17, 18, 24, 8, 21]"
7,15.0622,0.674071,69107.0,0.0767632,0.0,69107.0,15.001,0.87686,85358.0,0.0209465,0.0,69107.0,Modelo4,"[14, 9, 7, 28, 8, 21, 2, 19, 18, 3, 6, 5, 26, 16, 23, 29, 11, 4, 20, 32, 1, 31, 12, 24, 22, 30, 10, 15, 27, 13, 25, 17]"
8,15.0785,0.558184,80391.0,15.0046,0.0506273,80391.0,15.0005,0.847725,90527.0,0.0169258,0.0,76573.0,Modelo4,"[12, 9, 7, 27, 8, 19, 2, 17, 16, 3, 6, 5, 25, 14, 21, 29, 28, 4, 18, 32, 1, 31, 10, 23, 20, 30, 22, 13, 26, 11, 24, 15]"
